In [ ]:
# This notebook is derived from Qiskit and includes modifications by qBraid.
#
# (C) Copyright IBM 2020.
# (C) Copyright qBraid 2024.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

# qBraid-SDK IBM Batch Jobs Demo: Grover's Algorithm


In [ ]:
import qbraid

qbraid.__version__

## Creating the Circuits

In [ ]:
import numpy as np

from qiskit import QuantumCircuit

In [ ]:
%pip install pylatexenc  #Install pylatexenc to use the "mpl" circuit visualization sytle.

The code for this circuit was taken from IBMs [Qiskit Textbook](https://learn.qiskit.org/course/ch-algorithms/grovers-algorithm). Grover's algorithm let's us find a marked item in a box in √N steps as opposed to N steps classicaly. In this case we'll run Grover's algorithm for various numbers of steps to observe how the performance varies.

In [ ]:
n = 3
marked_entry = np.random.randint(0, 2**n - 1)
print("marked entry: ", marked_entry)

In [ ]:
# Circuit from qiskit textbook
def initialize_s(qc, qubits):
    """Apply a H-gate to 'qubits' in qc"""
    for q in qubits:
        qc.h(q)
    return qc


def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits - 1)
    qc.mcx(list(range(nqubits - 1)), nqubits - 1)  # multi-controlled-toffoli
    qc.h(nqubits - 1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s


qc = QuantumCircuit(3)
qc.cz(0, 2)
qc.cz(1, 2)
oracle_ex3 = qc.to_gate()
oracle_ex3.name = "U$_\omega$"

grover_circuit = QuantumCircuit(n)

grover_circuit.append(oracle_ex3, [0, 1, 2])
grover_circuit.append(diffuser(n), [0, 1, 2])

grover_circuit.draw("mpl")

In [ ]:
# create list of circuits to run
n_steps = 3
grover_init = QuantumCircuit(grover_circuit.num_qubits, grover_circuit.num_qubits)
grover_init.h(range(n))  # add the Hadamards
circuits = [grover_init]  # circuits[j] will have (VW)**j:
for _ in range(n_steps):
    circuits.append(circuits[-1].compose(grover_circuit))
for grover in circuits:
    grover.measure(range(n), range(n))  # add measurements

In [ ]:
from qbraid.visualization import circuit_drawer

circuit_drawer(circuits[3], output="mpl")

Now that we've created our Grover circuit for n = 0, 1, 2, and 3, we can simply pass the `circuits` array

# Running the Circuits

Now let's load in our IBM account. Note that you'll have to have IBM credentials to run this notebook. You can follow the instructions [here](https://github.com/Qiskit/qiskit-ibm-provider#provider-setup) to set them up.

Check which devices are online:

In [ ]:
from qbraid.runtime.ibm import QiskitRuntimeProvider

provider = QiskitRuntimeProvider("YOUR_API_KEY")
provider.get_devices()

In this tutorial we'll use IBM's Kyiv computer, since we see that it's online. Now we can use qBraid's `QuantumDevice` to run this job. The device wrapper adds a layer of abstraction, allowing us to run more types of circuits on more devices, and in this case letting us use the `run_batch` method. 

Batch experiments combine individual experiments on any subset of qubits into a single composite experiment which appends all the circuits from each component experiment into a single batch of circuits to be executed as one experiment job. This is useful because it allows us to run multiple circuits in parallel. It is especially useful for situations like this, where we want to run many similar circuits and collectively analyze their results

In [ ]:
device = provider.get_device("ibm_kyiv")

print(device)

In [ ]:
job = device.run(circuits, shots=100)

In [ ]:
job.status()

Since it takes awhile to run, we grab the id directly after it finishes. You can use the [qBraid API](https://docs.qbraid.com/api-reference/api-reference/get-quantum-jobs#get-jobs) to find the job ID for your job. 

In [ ]:
from qbraid.runtime.ibm import QiskitJob

job = QiskitJob("YOUR_JOB_ID", device=device)

job.status()

In [ ]:
batch_result = job.result()
batch_counts = batch_result.data.get_counts() 
# this notebook calls __version__ but doesn't specify...
# I changed it but we can change back if needed

for count in batch_counts:
    print(count)

We see that our results line up roughly with the theoretical prediction in the textbook. With 0 grover steps, the probability is basically evenly distributed. At one step, we see it is roughly 80% correct, as we expect. The probability then peaks at 2 steps and dips again at 3 steps.